<a href="https://colab.research.google.com/github/AtogmatomaProgramming/OdonScanAI/blob/develop/manage_model_v_4_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#=== Elementos a importar ===

import tensorflow as tf
from keras import layers, models
import os
import numpy as np
import cv2
import random
from sklearn.preprocessing import LabelEncoder

In [ ]:
# === Conectar con la carpeta de Drive ===
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# === Procesado del etiquetado de imágenes ===
width = 224
height = 224
ruta_train = '/content/drive/My Drive/alpha_training/frames/'


train_x = []
train_y = []

labels = os.listdir(ruta_train)
encoder = LabelEncoder()

# Codificación de etiquetas
encoded_labels = encoder.fit_transform(labels)
print("Clases codificadas:", dict(zip(labels, encoded_labels)))

print(labels)

for i in os.listdir(ruta_train):
    for j in os.listdir(os.path.join(ruta_train, i)):
        img = cv2.imread(ruta_train+i+'/'+j)
        resized_image = cv2.resize(img, (width, height))

        train_x.append(resized_image)

        for x,y in enumerate(labels):
            if y == i:
                array = np.zeros(len(labels))
                array[x]=1
                train_y.append(array)

x_data = np.array(train_x)
y_data = np.array(train_y)


Clases codificadas: {'elemento_desconocido': 0, 'lepidorhombus_whiffiagonis': 1, 'micromesistius_poutassou': 2}
['elemento_desconocido', 'lepidorhombus_whiffiagonis', 'micromesistius_poutassou']


In [ ]:
# === Desarrollo del Modelo ===

model = tf.keras.Sequential([
    layers.Conv2D(32, 3,3, input_shape=(width, height, 3)),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(32, 3,3),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(64, 3,3),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Flatten(),
    layers.Dense(64),
    layers.Activation('relu'),
    layers.Dropout(0.5),
    layers.Dense(len(labels)),
    layers.Activation('sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# === Entrenar modelo ===

epochs = 100

model.fit(x_data, y_data, epochs = epochs)

models.save_model(model, '/content/drive/My Drive/alpha_training/modelo_gamma.keras')

In [23]:
# === Testeo modelo ===

model = models.load_model('/content/drive/My Drive/alpha_training/modelo_gamma.keras')

ruta_predict = '/content/drive/My Drive/alpha_training/foto_test/lirio_2.jpg'
my_image = cv2.imread(ruta_predict)
my_image = cv2.resize(my_image, (width, height))

result = model.predict(np.array([my_image]))[0]

porcentaje = max(result)*100

grupo = labels[result.argmax()]

print(grupo, round(porcentaje))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
micromesistius_poutassou 100
